In [ ]:
"""
Making figures for current simulation plots
"""
import numpy as np

import complete_edl as edl
import constants as C

DEFAULT_GAMMA = 4
DEFAULT_CONC = 1e-1  # 100 mM
GAMMA_RANGE = [3, 4, 5, 6]
CONC_RANGE = [10e-3, 100e-3, 1000e-3]

C_1 = 6
C_2 = 3


def calculate_current(gold_sol, silanol_sol):
    """
    Calculate the current from a Aqueous models solved for gold and insulator
    """
    frac_sioh = (
        C.N_SITES_SILICA
        + silanol_sol["efield"][0] * silanol_sol["eps"][0] * C.EPS_0 / C.E_0
    ) / C.N_SITES_SILICA
    reorganization = C_1 * C.E_0 + np.abs(silanol_sol["pressure"][0]) * C_2 / au.n_max
    reaction_energy = C.E_0 * gold_sol["phi0"]
    e_act = (reorganization + reaction_energy) ** 2 / (4 * reorganization)
    return -frac_sioh * np.exp(-C.BETA * e_act)


PH_WERT_GAMM = 13
potentials_v_rhe_gamm = np.linspace(-0.5, 0, 100)
potentials_gamm = potentials_v_rhe_gamm - C.AU_PZC_SHE_V - 59e-3 * PH_WERT_GAMM
current_gamm = np.zeros((len(GAMMA_RANGE), len(potentials_gamm)))

for i, gamma in enumerate(GAMMA_RANGE):
    au = edl.Aqueous(DEFAULT_CONC, gamma, DEFAULT_GAMMA, DEFAULT_GAMMA, C_2)
    au_sol = au.potential_sweep(potentials_gamm, p_h=PH_WERT_GAMM)
    silica = edl.Aqueous(DEFAULT_CONC, gamma, DEFAULT_GAMMA, DEFAULT_GAMMA, C_2)
    silica_sol = silica.insulator_spatial_profiles(p_h=PH_WERT_GAMM, tol=1e-2)

    current_gamm[i, :] = calculate_current(au_sol, silica_sol)
current_gamm = current_gamm / np.max(np.abs(current_gamm))

PH_WERT_CONC = 7
potentials_v_rhe_conc = np.linspace(-0.25, 0, 100)
potentials_conc = potentials_v_rhe_conc - C.AU_PZC_SHE_V - 59e-3 * PH_WERT_CONC
current_conc = np.zeros((len(CONC_RANGE), len(potentials_conc)))

for i, conc in enumerate(CONC_RANGE):
    au = edl.Aqueous(conc, DEFAULT_GAMMA, DEFAULT_GAMMA, DEFAULT_GAMMA, C_2)
    au_sol = au.potential_sweep(potentials_conc, p_h=PH_WERT_CONC)
    silica = edl.Aqueous(conc, DEFAULT_GAMMA, DEFAULT_GAMMA, DEFAULT_GAMMA, C_2)
    silica_sol = silica.insulator_spatial_profiles(p_h=PH_WERT_CONC, tol=1e-2)
    current_conc[i, :] = calculate_current(au_sol, silica_sol)
current_conc = current_conc / np.max(np.abs(current_conc))

In [ ]:
import pandas as pd
conc_df = pd.DataFrame(
    data=current_conc.T,
    index=potentials_v_rhe_conc,
    columns=[f'conc = {conc*1e3:.0f} mM' for conc in CONC_RANGE]
)
conc_df.index.name='Potential vs. RHE at pH 7'
conc_df.to_csv('current_conc.csv')


In [ ]:
import pandas as pd
gamm_df = pd.DataFrame(
    data=current_gamm.T,
    index=potentials_v_rhe_gamm,
    columns=[f'gamma = {gamma}' for gamma in GAMMA_RANGE]
)
gamm_df.index.name='Potential vs. RHE at pH 13'
gamm_df.to_csv('current_gamma.csv')
